# Full & Reduced model with protocol MIN_SYN_BURST

Run the protocol MIN_SYN_BURST with the full Gillies & Willshaw (2005) STN model, then reduce the model according to our BranchZip algorithm, then rerun protocol and compare responses

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# Enable connecting with ipyton console --existing
# %connect_info

# print code version (hash of checked out version)
!git log -1

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..
%pwd

In [ ]:
# Import all our functions
from evalmodel.stn_model_evaluation import *

# print synapse info
def print_synapse_info(evaluator, model):
    """
    Print all synapse and NetCon parameter values
    """
    
    for pre_pop, mech in [(Pop.CTX, 'GLUsyn'), (Pop.GPE, 'GABAsyn')]:

        input_dict = evaluator.get_inputs(pre_pop, model)
        syn_param_names = cpd.getSynMechParamNames(mech)

        for nc in input_dict['syn_NetCons']:
            syn = nc.syn()
            print '\nInfo for synapse {} @ {}'.format(syn, syn.get_segment())
            print 'NetCon.weight[0] : {}'.format(nc.weight[0])

            for param_name in syn_param_names:
                print '{} : {}'.format(param_name, getattr(syn, param_name, 'not found'))

## Build full model

Build the Gillies & Willshaw STN model

In [ ]:
full_model = StnModel.Gillies2005
red_model = StnModel.Gillies_FoldMarasco

# Make cell model and evaluator
evaluator = StnModelEvaluator(full_model, PhysioState.NORMAL)
evaluator.build_cell(full_model)

## Run full model

Set up full model, and run the protocol

Explanation of plots:

- the first plot is the membrane voltage in several compartments (soma, a distal segment, and in the compartments where the GLU and GABA synapses are located respectively)


- the second plot are variables of the GLU and GABA synapse mechanism, respectively:
    + GLU synapse, first trace: AMPA conductance
    + GLU synapse, second trace: NMDA synapse
    + third trace: corresponds approximately to the size of the Readily Releasable Pool of vesicles
    + fourth trace: corresponds roughly the fraction of available vesicles released upon each spike
    
    
- the third plot are variables of the GABA-A + GABA-B synapse mechanism
    + first trace is GABA-A (ionotropic)
    + second trace is GABA-B (metabotropic)
    + the other two traces are same as GLU synapse

In [ ]:
# Run protocol
proto = StimProtocol.MIN_SYN_BURST
evaluator.setup_run_protocol(proto)
evaluator.plot_traces(proto)

## Reduce model

Collapse dendritic trees using BranchZip algorithm, then remap synapses

In [ ]:
# Model reduction
evaluator.build_cell(red_model)
evaluator.target_model = red_model

In [ ]:
print_synapse_info(evaluator, red_model)

## Run reduced model

Run the same protocol with the reduced model.


In [ ]:
# Run Protocol
evaluator.setup_run_protocol(proto)
evaluator.plot_traces(proto)